In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 44.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=9c02598ea1b9bb2eb2227b7426e9fd7d981cf719e835231e1f6dcc8d3bd7085f
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:

from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("DataExplore") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","4g") \
.getOrCreate()

In [4]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("/content/simple_data.csv")

In [5]:
df.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir
0,1,Cemal,35,Isci,Ankara,3500
1,2,Ceyda,42,Memur,Kayseri,4200
2,3,Timur,30,Müzisyen,Istanbul,9000
3,4,Burcu,29,Pazarlamaci,Ankara,4200
4,5,Yasemin,23,Pazarlamaci,Bursa,4800


In [6]:
from pyspark.sql.functions import *

df1 = df.withColumn("ekonomik_durum",
when(col("aylik_gelir") >= 7000, "iyi").otherwise("kötü")
)

df1.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum
0,1,Cemal,35,Isci,Ankara,3500,kötü
1,2,Ceyda,42,Memur,Kayseri,4200,kötü
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü


### **🔍StringIndexer**

In [31]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler

In [10]:
meslek_indexer = StringIndexer() \
.setInputCol("meslek") \
.setOutputCol("meslek_index") \
.setHandleInvalid("skip") 

meslek_indexer_model = meslek_indexer.fit(df1)
meslek_idexer_df = meslek_indexer_model.transform(df1)

In [11]:
#en cok tekrarlanana sifir vererek ardisik sekilde index tanimliyor basliyor.

meslek_idexer_df.toPandas().head(15)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0
5,6,Ali,33,Memur,Ankara,4250,kötü,0.0
6,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,2.0
7,8,Murat,31,Müzisyen,Istanbul,12000,iyi,1.0
8,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0
9,10,Muhittin,46,Berber,Istanbul,12000,iyi,4.0


In [12]:
df1.groupBy(col("meslek")) \
.agg(count("*").alias("sayi")) \
.sort(desc("sayi")) \
.toPandas().head()

,meslek,sayi
0,Memur,3
1,Müzisyen,3
2,Pazarlamaci,3
3,Doktor,2
4,Isci,1


In [13]:
sehir_indexer = StringIndexer() \
.setInputCol("sehir") \
.setOutputCol("sehir_index")

sehir_indexer_model = sehir_indexer.fit(meslek_idexer_df)
sehir_indexer_df = sehir_indexer_model.transform(meslek_idexer_df)

In [14]:
sehir_indexer_df.toPandas().head(15)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0
5,6,Ali,33,Memur,Ankara,4250,kötü,0.0,0.0
6,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,2.0,1.0
7,8,Murat,31,Müzisyen,Istanbul,12000,iyi,1.0,1.0
8,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0,0.0
9,10,Muhittin,46,Berber,Istanbul,12000,iyi,4.0,1.0


In [21]:
df1.groupBy("sehir") \
.agg(count("sehir").alias("sayi")) \
.sort(desc("sayi")) \
.toPandas().head(20)

,sehir,sayi
0,Ankara,7
1,Istanbul,4
2,Çorum,1
3,Kayseri,1
4,Bursa,1
5,İzmir,1


### 🔍OneHotEncoder

In [18]:
encoder = OneHotEncoder() \
.setInputCols(["meslek_index","sehir_index"]) \
.setOutputCols(["meslek_encoded","sehir_encoded"])

encoder_model = encoder.fit(sehir_indexer_df)
encoder_df = encoder_model.transform(sehir_indexer_df)


In [22]:
encoder_df.toPandas().head(5)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)"
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"


## 🔍VectorAssembler

In [24]:
assembler = VectorAssembler() \
.setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
.setOutputCol("vectorized_features")

In [26]:
vector_df = assembler.transform(encoder_df)
vector_df.select("vectorized_features").toPandas().head(15)

,vectorized_features
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)"
2,"(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
3,"(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
4,"(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)"
5,"(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
6,"(29.0, 7300.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
7,"(31.0, 12000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
8,"(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
9,"(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"


### 🔍LabelIndexer

In [27]:
label_indexer = StringIndexer() \
.setInputCol("ekonomik_durum") \
.setOutputCol("label")

In [28]:
label_indexer_model = label_indexer.fit(vector_df)
label_indexer_df = label_indexer_model.transform(vector_df)

In [30]:
label_indexer_df.select("vectorized_features","label").toPandas().head(5)


,vectorized_features,label
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",0.0
2,"(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
3,"(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
4,"(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0


### 🔍StandartScaler

In [32]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

In [33]:
scaler_model = scaler.fit(label_indexer_df)
scaler_df = scaler_model.transform(label_indexer_df)

In [34]:
scaler_df.select("features","label").toPandas().head()

,features,label
0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)",0.0
1,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)",0.0
2,"(4.292812263480104, 1.9859783574511787, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)",1.0
3,"(4.149718521364101, 0.9267899001438834, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)",0.0
4,"(3.29115606866808, 1.0591884573072952, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)",0.0


### 🔍Train-Test Split

In [36]:
train_df, test_df = scaler_df.randomSplit([0.8,0.2], seed=142)

In [43]:
train_df.toPandas().head(3)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",0.0,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)"
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.292812263480104, 1.9859783574511787, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"


In [42]:
print(train_df.count())
print(test_df.count())

11
4


### 🔍Model

In [44]:
from pyspark.ml.classification import LogisticRegression

In [49]:
lr_object = LogisticRegression() \
.setFeaturesCol("features") \
.setLabelCol("label") \
.setPredictionCol("prediction")

In [50]:
lr_model = lr_object.fit(train_df)

In [51]:
result_df = lr_model.transform(test_df)

In [52]:
result_df.select("label","prediction").toPandas().head()

,label,prediction
0,0.0,0.0
1,1.0,1.0
2,0.0,0.0
3,1.0,0.0
